## Advanced Graphs

In this notebook we will illustrate the different types of microservices that can be deployed in Seldon:
* Model
* Transformer
* Router
* Combiner
* Output Transformer

We will deploy graphs of increasing complexity. But first we need to install seldon on the cluster.

In [ ]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

In [ ]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd

In [ ]:
!kubectl create namespace graphs

In [ ]:
!helm install ../helm-charts/seldon-core --name seldon-core \
        --set cluster_manager.rbac=true \
        --set cluster_manager_service_type=LoadBalancer \
        --set apife_service_type=LoadBalancer \
        --namespace graphs

In [ ]:
!kubectl get svc -n seldon seldon-apiserver -n graphs

Setup pyhton code to do RESR and gRPC requests. **Only run this when the LoadBalancer created by GCP for the seldon-apife is running**

In [ ]:
from visualizer import get_graph
import json

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
from proto import prediction_pb2
from proto import prediction_pb2_grpc
import grpc
import commands

NAMESPACE="graphs"
SELDON_API_IP=commands.getoutput("kubectl get svc -n "+NAMESPACE+" seldon-apiserver -o jsonpath='{.status.loadBalancer.ingress[0].ip}'")

def get_token():
    payload = {'grant_type': 'client_credentials'}
    response = requests.post(
                "http://{}:8080/oauth/token".format(SELDON_API_IP),
                auth=HTTPBasicAuth('oauth-key', 'oauth-secret'),
                data=payload)
    token =  response.json()["access_token"]
    return token

def rest_request():
    token = get_token()
    headers = {'Authorization': 'Bearer '+token}
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://{}:8080/api/v0.1/predictions".format(SELDON_API_IP),
                headers=headers,
                json=payload)
    print response.text
    
def grpc_request():
    token = get_token()
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = [3,2],
                values = [1.0,1.0,2.0,3.0,4.0,5.0]
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel("{}:5000".format(SELDON_API_IP))
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('oauth_token', token)]
    response = stub.Predict(request=request,metadata=metadata)
    print response

In [ ]:
!kubectl get pods -n graphs

## Simple Model

In [ ]:
get_graph("resources/model.json")

First we will check that everything works by running a simple model

In [ ]:
!kubectl apply -f resources/model.json -n graphs

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath="{.status}" -n graphs

In [ ]:
rest_request()

In [ ]:
grpc_request()

In [ ]:
!kubectl delete -f resources/model.json -n graphs

## Random AB Test

In [ ]:
get_graph("resources/random_ab_test.json")

In this example we will deploy 2 models under an AB test router. The Random AB Test we will use is implemented directly in Seldon, this is not a microservice, so no docker image needs to be specified.

The json graph is as follows:

In [ ]:
json.load(open("./resources/random_ab_test.json",'r')).get("spec").get("predictors")[0].get("graph")

We specify ``` "implementation": "RANDOM_ABTEST" ``` to get the AB Test router implemented in Seldon.
We pass the parameter ratioA which corresponds to the ratio of requests that will be passed to the first child of the Router

In [ ]:
!kubectl apply -f resources/random_ab_test.json -n graphs

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' -n graphs

In [ ]:
rest_request()

In [ ]:
grpc_request()

In [ ]:
!kubectl delete -f resources/random_ab_test.json -n graphs

## Average Combiner

In [ ]:
get_graph("resources/ensemble.json")

In this example again we will use a service implemented in seldon, called Average Combiner. It takes the outputs of several models and returns the arithmetic mean of them.

The json is as follows:

In [ ]:
json.load(open("./resources/ensemble.json",'r')).get("spec").get("predictors")[0].get("graph")

In [ ]:
!kubectl apply -f resources/ensemble.json -n graphs

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' -n graphs

In [ ]:
rest_request()

In [ ]:
grpc_request()

In [ ]:
!kubectl delete -f resources/ensemble.json -n graphs

## Feature Transformer

In [ ]:
get_graph("resources/feature_transform.json")

In this example we deploy a simple model under a feature transformation microservice. For the transformer we will use the docker image seldonio/mock_transformer:1.0

Since this is not implemented in Seldon, the type of predictive unit (TRANSFORMER) needs to be specified in the graph so that Seldon Core knows which API this microservice implements.

The json is as follows:

In [ ]:
json.load(open("./resources/feature_transform.json",'r')).get("spec").get("predictors")[0].get("graph")

In [ ]:
!kubectl apply -f resources/feature_transform.json -n graphs

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' -n graphs

In [ ]:
rest_request()

In [ ]:
grpc_request()

In [ ]:
!kubectl delete -f resources/feature_transform.json -n graphs

## Outlier Detector

In [ ]:
get_graph("resources/outlier_detector.json")

In this example we will have different four components: 
* A transformer, the outlier detector
* A router, the random AB test
* Two models

The outlier detector is a special kind of transformer that will populate a tag in the response metadata with the outlier score it has calculated. 
We use the docker image seldonio/mock_outlier_detector:1.0 for the outlier detector.

The json is as follows:

In [ ]:
json.load(open("./resources/outlier_detector.json",'r')).get("spec").get("predictors")[0].get("graph")

In [ ]:
!kubectl apply -f resources/outlier_detector.json -n graphs

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' -n graphs

In [ ]:
rest_request()

In [ ]:
grpc_request()

In [ ]:
!kubectl delete -f resources/outlier_detector.json -n graphs

## Complex Graph

In [ ]:
get_graph("resources/complex_graph.json")

In this final example we will deploy a complex graph with all of the components that have been used so far.

In [ ]:
!kubectl apply -f resources/complex_graph.json -n graphs

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' -n graphs

In [ ]:
rest_request()

In [ ]:
grpc_request()

In [ ]:
!kubectl delete -f resources/complex_graph.json -n graphs

## Tear Down

In [ ]:
!helm delete seldon-core --purge

In [ ]:
!helm delete seldon-core-crd --purge